In [264]:
import pandas as pd
import re
import requests
import difflib
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [289]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get('https://fantasyfootballcalculator.com/adp', headers=header)
ffc = pd.read_html(r.text, header = 0); ffc = ffc[0]

ffc = ffc[['Pick', 'Name', 'Overall', 'Std.Dev', 'High', 'Low']]
ffc = ffc.rename(columns = {'Name': 'name'})



fp = pd.read_html('https://www.fantasypros.com/nfl/adp/overall')[0]
fp['Player Team (Bye)'] = fp.iloc[:,1].astype('str')
name = []
for i in range(0,len(fp)):
    try: 
        name.append (re.findall('(.*?)\s[A-Z][A-Z]', fp.iloc[i,1])[0])
    except:
        name.append("")
    

fp['name'] = name
fp = fp.drop('Player Team (Bye)', axis = 1)

stopwords = [" Jr.", " II", " III", " IV", "Defense"]



fp['name'].apply(lambda x: [item for item in x if item not in stopwords])

pat = r'\b(?:{})\b'.format('|'.join(stopwords))
fp['name'] = fp['name'].str.replace(pat, '')
ffc['name'] = ffc['name'].str.replace(pat, '')
fp['name'] = fp['name'].str.replace('.', '')
ffc['name'] = ffc['name'].str.replace('.', '')


fp['name'] = fp['name'].str.upper()
ffc['name'] = ffc['name'].str.upper()

rem_name = []
remainder = ffc[-ffc['name'].isin(fp['name'])]
for i in range(0, len(remainder)):
    rem_name.append( difflib.get_close_matches(remainder.iloc[i,1], fp['name'], cutoff = .55)[0])

indexer = ffc[-ffc['name'].isin(fp['name'])].index
ffc.loc[indexer, "name"] = rem_name



df = ffc.merge(fp)

df['test_name'] = ""
rem_name = []
for i in range(0, len(df)):
    rem_name.append( difflib.get_close_matches(df.iloc[i,1], player_names, cutoff = .55)[0])

df["test_name"] = rem_name


df.to_csv('../adp.csv')


In [270]:
player_names = pd.read_csv('../player_list.csv')
player_names = player_names['x']

In [271]:
player_names

0                SAQUON BARKLEY
1           CHRISTIAN MCCAFFREY
2                  ALVIN KAMARA
3               EZEKIEL ELLIOTT
4                 DAVID JOHNSON
5               DEANDRE HOPKINS
6                 DAVANTE ADAMS
7                  JAMES CONNER
8                   JULIO JONES
9                    NICK CHUBB
10                  TODD GURLEY
11                 TRAVIS KELCE
12          JUJU SMITH-SCHUSTER
13                    JOE MIXON
14                ODELL BECKHAM
15                  DALVIN COOK
16                  TYREEK HILL
17                 LE'VEON BELL
18               MICHAEL THOMAS
19                   MIKE EVANS
20                  T.Y. HILTON
21              KERRYON JOHNSON
22                 CHRIS CARSON
23                 KEENAN ALLEN
24                 ADAM THIELEN
25                GEORGE KITTLE
26            LEONARD FOURNETTE
27                  MARLON MACK
28                DERRICK HENRY
29              DEVONTA FREEMAN
30                 ROBERT WOODS
31      

In [272]:
df

,Pick,name,Overall,Std.Dev,High,Low,Rank,POS,CBS,FFC,Fantrax,AVG
0,1.01,SAQUON BARKLEY,1.5,0.7,1.01,1.04,1,RB1,1.0,1.0,1.0,1.0
1,1.02,ALVIN KAMARA,2.4,0.8,1.01,1.05,2,RB2,2.0,2.0,3.0,2.3
2,1.03,CHRISTIAN MCCAFFREY,2.5,0.9,1.01,1.05,3,RB3,3.0,3.0,2.0,2.7
3,1.05,EZEKIEL ELLIOTT,4.5,1.4,1.01,1.11,4,RB4,4.0,4.0,4.0,4.0
4,1.05,DAVID JOHNSON,5.2,1.3,1.03,1.11,6,RB5,8.0,5.0,5.0,6.0
5,1.06,DEANDRE HOPKINS,6.0,1.8,1.01,1.12,5,WR1,5.0,6.0,6.0,5.7
6,1.07,LE'VEON BELL,7.3,2.3,1.01,2.05,7,RB6,6.0,7.0,7.0,6.7
7,1.08,DAVANTE ADAMS,7.9,2.0,1.01,2.02,8,WR2,10.0,8.0,8.0,8.7
8,1.09,NICK CHUBB,8.5,2.3,1.02,2.05,14,RB10,19.0,9.0,14.0,14.0
9,1.10,JAMES CONNER,9.8,2.2,1.04,2.05,9,RB7,9.0,10.0,9.0,9.3


In [288]:
df['test_name'] = ""
rem_name = []
for i in range(0, len(df)):
    rem_name.append( difflib.get_close_matches(df.iloc[i,1], player_names, cutoff = .55)[0])

df["test_name"] = rem_name


In [284]:
len(rem_name)

201

In [285]:
len(df)

201

In [286]:
df['poop'] = rem_name

In [287]:
df

,Pick,name,Overall,Std.Dev,High,Low,Rank,POS,CBS,FFC,Fantrax,AVG,test_name,poop
0,1.01,SAQUON BARKLEY,1.5,0.7,1.01,1.04,1,RB1,1.0,1.0,1.0,1.0,,SAQUON BARKLEY
1,1.02,ALVIN KAMARA,2.4,0.8,1.01,1.05,2,RB2,2.0,2.0,3.0,2.3,,ALVIN KAMARA
2,1.03,CHRISTIAN MCCAFFREY,2.5,0.9,1.01,1.05,3,RB3,3.0,3.0,2.0,2.7,,CHRISTIAN MCCAFFREY
3,1.05,EZEKIEL ELLIOTT,4.5,1.4,1.01,1.11,4,RB4,4.0,4.0,4.0,4.0,,EZEKIEL ELLIOTT
4,1.05,DAVID JOHNSON,5.2,1.3,1.03,1.11,6,RB5,8.0,5.0,5.0,6.0,,DAVID JOHNSON
5,1.06,DEANDRE HOPKINS,6.0,1.8,1.01,1.12,5,WR1,5.0,6.0,6.0,5.7,,DEANDRE HOPKINS
6,1.07,LE'VEON BELL,7.3,2.3,1.01,2.05,7,RB6,6.0,7.0,7.0,6.7,,LE'VEON BELL
7,1.08,DAVANTE ADAMS,7.9,2.0,1.01,2.02,8,WR2,10.0,8.0,8.0,8.7,,DAVANTE ADAMS
8,1.09,NICK CHUBB,8.5,2.3,1.02,2.05,14,RB10,19.0,9.0,14.0,14.0,,NICK CHUBB
9,1.10,JAMES CONNER,9.8,2.2,1.04,2.05,9,RB7,9.0,10.0,9.0,9.3,,JAMES CONNER


In [290]:
df.sort_values('Rank')

,Pick,name,Overall,Std.Dev,High,Low,Rank,POS,CBS,FFC,Fantrax,AVG,test_name
0,1.01,SAQUON BARKLEY,1.5,0.7,1.01,1.04,1,RB1,1.0,1.0,1.0,1.0,SAQUON BARKLEY
1,1.02,ALVIN KAMARA,2.4,0.8,1.01,1.05,2,RB2,2.0,2.0,3.0,2.3,ALVIN KAMARA
2,1.03,CHRISTIAN MCCAFFREY,2.5,0.9,1.01,1.05,3,RB3,3.0,3.0,2.0,2.7,CHRISTIAN MCCAFFREY
3,1.05,EZEKIEL ELLIOTT,4.5,1.4,1.01,1.11,4,RB4,4.0,4.0,4.0,4.0,EZEKIEL ELLIOTT
5,1.06,DEANDRE HOPKINS,6.0,1.8,1.01,1.12,5,WR1,5.0,6.0,6.0,5.7,DEANDRE HOPKINS
4,1.05,DAVID JOHNSON,5.2,1.3,1.03,1.11,6,RB5,8.0,5.0,5.0,6.0,DAVID JOHNSON
6,1.07,LE'VEON BELL,7.3,2.3,1.01,2.05,7,RB6,6.0,7.0,7.0,6.7,LE'VEON BELL
7,1.08,DAVANTE ADAMS,7.9,2.0,1.01,2.02,8,WR2,10.0,8.0,8.0,8.7,DAVANTE ADAMS
9,1.10,JAMES CONNER,9.8,2.2,1.04,2.05,9,RB7,9.0,10.0,9.0,9.3,JAMES CONNER
10,1.11,TODD GURLEY,11.1,2.9,1.02,2.10,10,RB8,15.0,11.0,10.0,12.0,TODD GURLEY


In [298]:
len(df[df['Rank'] <= 150])

150

In [299]:
14 * 12

168